In [7]:
from ultralytics import YOLO
import cv2
import numpy as np

# ================== Paths ==================
MODEL_PATH = r"D:\ERU\AitronixEnv\Fire Detection system\best.pt"
INPUT_VIDEO = r"D:\ERU\AitronixEnv\Fire Detection system\fire.mp4"
OUTPUT_VIDEO = r"D:\ERU\AitronixEnv\Fire Detection system\fire_output.mp4"

# ================== Config ==================
CONF_THRESH = 0.25
IMGSZ = 640

# ================== Load Model ==================
model = YOLO(MODEL_PATH)

# ================== Video IO ==================
cap = cv2.VideoCapture(INPUT_VIDEO)
if not cap.isOpened():
    raise RuntimeError("❌ Cannot open video")

fps = cap.get(cv2.CAP_PROP_FPS)
fps = fps if fps > 0 else 25.0

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))

# ================== Display ==================
cv2.namedWindow("Output", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Output", 1280, 720)

print("🚀 Starting inference...")

# ================== Colors ==================
COLORS = {
    "fire": (0, 0, 255),    # 🔴 Red
    "smoke": (255, 0, 0)    # 🔵 Blue
}

# ================== Loop ==================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(
        frame,
        imgsz=IMGSZ,
        conf=CONF_THRESH,
        stream=True,
        verbose=False
    )

    for r in results:
        if r.boxes is None:
            continue

        boxes = r.boxes.xyxy.cpu().numpy().astype(int)
        confs = r.boxes.conf.cpu().numpy()
        clss = r.boxes.cls.cpu().numpy().astype(int)

        for (x1, y1, x2, y2), conf, cls in zip(boxes, confs, clss):
            class_name = model.names[cls]
            color = COLORS.get(class_name, (0, 255, 0))

            label = f"{class_name.upper()} {conf:.2f}"

            # ===== Text size =====
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.55
            thickness = 1

            (tw, th), baseline = cv2.getTextSize(label, font, font_scale, thickness)

            # ===== Label background (inside box) =====
            cv2.rectangle(
                frame,
                (x1, y1),
                (x1 + tw + 6, y1 + th + baseline + 6),
                color,
                -1
            )

            # ===== Label text =====
            cv2.putText(
                frame,
                label,
                (x1 + 3, y1 + th + 3),
                font,
                font_scale,
                (255, 255, 255),
                thickness,
                cv2.LINE_AA
            )

            # ===== Bounding box =====
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    out.write(frame)
    cv2.imshow("Output", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# ================== Cleanup ==================
cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Done. Saved to:", OUTPUT_VIDEO)


🚀 Starting inference...
✅ Done. Saved to: D:\ERU\AitronixEnv\Fire Detection system\fire_output.mp4
